## 3. Running Step by Step

In this running step by step, we will show how the ``openTEPES_run`` function works.

#### Path and case study
In this tutorial the main scripts of the model are located in DIR and the case study will be the ``9n`` that is located in the folder with the same name.
In addition, we also declare the ``SolverName`` and ``IndLogConsole``.

In [ ]:
import os
os._exit(00)

DirName = '/srv/conda/envs/notebook/lib/python3.8/site-packages/openTEPES'
CaseName = '9n'
SolverName = 'cbc'
IndLogConsole = 'No'

#### Libraries
We start by importing the relevant libraries.

In [ ]:
import time
import os

from pyomo.environ import ConcreteModel, Set
from openTEPES import openTEPES as oT

#### Data transforming
We start by importing the relevant libraries.

In [ ]:
InitialTime = time.time()

idxDict = dict()
idxDict[0    ] = 0
idxDict[0.0  ] = 0
idxDict['No' ] = 0
idxDict['NO' ] = 0
idxDict['no' ] = 0
idxDict['N'  ] = 0
idxDict['n'  ] = 0
idxDict['Yes'] = 1
idxDict['YES'] = 1
idxDict['yes'] = 1
idxDict['Y'  ] = 1
idxDict['y'  ] = 1

pIndLogConsole = [j for i, j in idxDict.items() if i == IndLogConsole]

#### Model object definition
We use the ``ConcreteModel`` function from pyomo to define our model, as follows:

In [ ]:
mTEPES = ConcreteModel('openTEPES')

#### Define sets and parameters

In [ ]:
oT.InputData(DirName, CaseName, mTEPES, pIndLogConsole)

#### Define variables

In [ ]:
oT.SettingUpVariables(mTEPES, mTEPES)

#### Investment model objective function

In [ ]:
oT.InvestmentModelFormulation(mTEPES, mTEPES, pIndLogConsole)

#### Iterative model formulation for each stage of a year
This part has two main sections: the first one active only scenario, period, and load levels to be used, and the second one, define the operation model objective function and constraints by stage.

In [ ]:
for sc,p,st in mTEPES.scc*mTEPES.pp*mTEPES.stt:
    # only scenario, period and load levels to formulate
    mTEPES.del_component(mTEPES.sc)
    mTEPES.del_component(mTEPES.p )
    mTEPES.del_component(mTEPES.st)
    mTEPES.del_component(mTEPES.n )
    mTEPES.del_component(mTEPES.n2)
    mTEPES.sc = Set(initialize=mTEPES.scc, ordered=True, doc='scenarios',   filter=lambda mTEPES,scc: scc in mTEPES.scc and sc == scc and mTEPES.pScenProb   [scc])
    mTEPES.p  = Set(initialize=mTEPES.pp,  ordered=True, doc='periods',     filter=lambda mTEPES,pp : pp  in                p  == pp                              )
    mTEPES.st = Set(initialize=mTEPES.stt, ordered=True, doc='stages',      filter=lambda mTEPES,stt: stt in mTEPES.stt and st == stt and mTEPES.pStageWeight[stt] and sum(1 for (st, nn) in mTEPES.s2n))
    mTEPES.n  = Set(initialize=mTEPES.nn,  ordered=True, doc='load levels', filter=lambda mTEPES,nn : nn  in mTEPES.pDuration and (st,nn) in mTEPES.s2n)
    mTEPES.n2 = Set(initialize=mTEPES.nn,  ordered=True, doc='load levels', filter=lambda mTEPES,nn : nn  in mTEPES.pDuration and (st,nn) in mTEPES.s2n)

    # operation model objective function and constraints by stage
    oT.GenerationOperationModelFormulation(mTEPES, mTEPES, pIndLogConsole, st)
    oT.NetworkSwitchingModelFormulation   (mTEPES, mTEPES, pIndLogConsole, st)
    oT.NetworkOperationModelFormulation   (mTEPES, mTEPES, pIndLogConsole, st)

#### Create lp-format file
We save the LP-format file in the folder called ``9n``. 

In [ ]:
StartTime         = time.time()
mTEPES.write(CaseName+'/openTEPES_'+CaseName+'.lp', io_options={'symbolic_solver_labels': True})
WritingLPFileTime = time.time() - StartTime
StartTime         = time.time()
print('Writing LP file                        ... ', round(WritingLPFileTime), 's')

#### Solving the mTEPES problem

In [ ]:
oT.ProblemSolving(DirName, CaseName, SolverName, mTEPES, mTEPES, pIndLogConsole)

#### Activating all possible scenario, period, and load levels

In [ ]:
mTEPES.del_component(mTEPES.sc)
mTEPES.del_component(mTEPES.p )
mTEPES.del_component(mTEPES.st)
mTEPES.del_component(mTEPES.n )
mTEPES.sc = Set(initialize=mTEPES.scc, ordered=True, doc='scenarios',   filter=lambda mTEPES,scc: scc in mTEPES.scc and mTEPES.pScenProb   [scc])
mTEPES.p  = Set(initialize=mTEPES.pp,  ordered=True, doc='periods',     filter=lambda mTEPES,pp : pp  in p  == pp                               )
mTEPES.st = Set(initialize=mTEPES.stt, ordered=True, doc='stages',      filter=lambda mTEPES,stt: stt in mTEPES.stt and mTEPES.pStageWeight[stt] and sum(1 for (stt, nn) in mTEPES.s2n))
mTEPES.n  = Set(initialize=mTEPES.nn,  ordered=True, doc='load levels', filter=lambda mTEPES,nn : nn  in                mTEPES.pDuration        )


#### Writing and plotting results

In [ ]:
oT.InvestmentResults         (DirName, CaseName, mTEPES, mTEPES)
oT.GenerationOperationResults(DirName, CaseName, mTEPES, mTEPES)
oT.ESSOperationResults       (DirName, CaseName, mTEPES, mTEPES)
oT.FlexibilityResults        (DirName, CaseName, mTEPES, mTEPES)
oT.NetworkOperationResults   (DirName, CaseName, mTEPES, mTEPES)
oT.MarginalResults           (DirName, CaseName, mTEPES, mTEPES)
oT.EconomicResults           (DirName, CaseName, mTEPES, mTEPES)
oT.NetworkMapResults         (DirName, CaseName, mTEPES, mTEPES)

TotalTime = time.time() - InitialTime
print('Total time                             ... ', round(TotalTime), 's')